In [2]:
# IMPORT REQUIRED PACKAGES
from PyPDF2 import PdfReader
import os
import pandas as pd
import re

In [75]:
# Store bill names and texts lists
bill_states, bill_names, bill_texts = [], [], []
# Set the initial starting path for the bill pdfs
path = "../raw_data/"
for filename in os.listdir(path):
    # Prep for opening the PDF
    filename_lower = filename.lower()
    if filename_lower.endswith(".pdf"):
        # Open the current PDF file
        pdf = open(os.path.join(path, filename), "rb")
        # Create a PDF reader object
        pdf_reader = PdfReader(pdf)
        # Make a list to store the text from each page in
        curr_lines = []
        # Loop through each page in the PDF document
        for page in range(pdf_reader.getNumPages()):
            # Extract the text from the current page
            page_text = pdf_reader.getPage(page).extractText()
            # Getting rid of non-ascii characters
            page_text = page_text.replace("â??", "")
            page_text = page_text.encode('ascii', 'ignore').decode('ascii')
            page_text = re.sub(r'[^\x00-\x7F]+', "", page_text)
            page_text = re.sub(r'[^a-zA-Z0-9]', "", page_text)
            # Add lines to list
            curr_lines += page_text.splitlines()
        # Add full bill text to list
        bill_texts.append(curr_lines)
        # Get the bill name and state and append to lists
        filename_split1 = filename.split("_")
        # Add state to state list
        bill_states.append(filename_split1[0])
        # Add bill names to list
        filename_split2 = filename_split1[1].split(".")
        bill_names.append(filename_split2[0])
        # Close the PDF file
        pdf.close()
# Make a dictionary
bills_dict = {"state": bill_states, "bill_name": bill_names, "text": bill_texts}
# Convert to data frame
bills_df = pd.DataFrame(bills_dict)
# Sort bills in order by state
bills_df = bills_df.sort_values("state").reset_index(drop = True)

Multiple definitions in dictionary at byte 0x6970 for key /Outlines
Multiple definitions in dictionary at byte 0x690a for key /Outlines
/Users/miamayerhofer/opt/anaconda3/envs/anly521/lib/python3.11/site-packages/PyPDF2/_cmap.py:134: PdfReadWarning: Advanced encoding /Identity#2DH not implemented yet
  warnings.warn(
Multiple definitions in dictionary at byte 0xcd39 for key /Outlines
Multiple definitions in dictionary at byte 0x68ba for key /Outlines
Multiple definitions in dictionary at byte 0x6acc for key /Outlines
Multiple definitions in dictionary at byte 0x6ace for key /Outlines
Multiple definitions in dictionary at byte 0x72b1 for key /Outlines


In [76]:
bills_df = bills_df.reset_index(drop = True)
# Make a csv file
bills_df.to_csv("../modified_data/bill_texts.csv")

In [85]:
# Import ACLU table
aclu_table = pd.read_csv("../modified_data/aclu_bill_data.csv")
# Remove space in bill names
aclu_table["bill_name"] = aclu_table["bill_name"].str.replace(" ", "")
aclu_table["bill_name"] = aclu_table["bill_name"].str.replace(".", "")
# Create a dictionary mapping state names to acronyms
state_dict = {
    'Alabama': 'AL', 'Alaska': 'AK','Arizona': 'AZ','Arkansas': 'AR',
    'California': 'CA','Colorado': 'CO','Connecticut': 'CT','Delaware': 'DE',
    'Florida': 'FL','Georgia': 'GA','Hawaii': 'HI','Idaho': 'ID','Illinois': 'IL','Indiana': 'IN',
    'Iowa': 'IA','Kansas': 'KS','Kentucky': 'KY','Louisiana': 'LA','Maine': 'ME','Maryland': 'MD',
    'Massachusetts': 'MA','Michigan': 'MI','Minnesota': 'MN','Mississippi': 'MS','Missouri': 'MO',
    'Montana': 'MT','Nebraska': 'NE','Nevada': 'NV','New Hampshire': 'NH','New Jersey': 'NJ',
    'New Mexico': 'NM','New York': 'NY','North Carolina': 'NC','North Dakota': 'ND','Ohio': 'OH',
    'Oklahoma': 'OK','Oregon': 'OR','Pennsylvania': 'PA','Rhode Island': 'RI','South Carolina': 'SC',
    'South Dakota': 'SD','Tennessee': 'TN','Texas': 'TX','Utah': 'UT','Vermont': 'VT','Virginia': 'VA',
    'Washington': 'WA','West Virginia': 'WV','Wisconsin': 'WI','Wyoming': 'WY'}
# Change state names to acronym
aclu_table["state"] = aclu_table["state"].map(state_dict)
aclu_table.loc[211, "bill_name"] = "SB228"

/var/folders/_p/d81k_b_93575z7h2220jh4cr0000gn/T/ipykernel_15229/4046696062.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  aclu_table["bill_name"] = aclu_table["bill_name"].str.replace(".", "")


In [86]:
aclu_table

,state,link,bill_name,category,status
0,AK,https://www.akleg.gov/basis/Bill/Detail/33?Roo...,HB27,Schools & Education,"Referred to committee, 01/19/2023"
1,AK,https://www.akleg.gov/basis/Bill/Detail/33?Roo...,HB105,Schools & Education,"First read and referred to committee, 03/08/2023"
2,AK,https://www.akleg.gov/basis/Bill/Detail/33?Roo...,SB96,Schools & Education,"First read and referred to committee, 03/08/2023"
3,AZ,https://apps.azleg.gov/BillStatus/BillOverview...,SB1028,Free Speech & Expression,"Passed Senate; House second read, 03/22/2023"
4,AZ,https://apps.azleg.gov/BillStatus/BillOverview...,SB1026,Free Speech & Expression,"Passed Senate; House second read, 03/09/2023"
...,...,...,...,...,...
429,WY,https://wyoleg.gov/Legislation/2023/SF0111,SF111,Healthcare,Passed Senate; House did not consider for intr...
430,WY,https://wyoleg.gov/Legislation/2023/SF0117,SF117,Schools & Education,Passed Senate; House did not consider for intr...
431,WY,https://wyoleg.gov/Legislation/2023/HB0262,HB262,Civil Rights,"Died in committee, 02/07/2023"
432,WY,https://wyoleg.gov/Legislation/2023/SF0133,SF133,Schools & Education,"Became act without Governor’s signature, 03/17..."


In [87]:
bills_df

,state,bill_name,text
0,AK,SB270,[Strickenlanguagewouldbedeletedfromandunderlin...
1,AK,HB27,[HB0027a1HB27NewTextUnderlinedDELETEDTEXTBRACK...
2,AK,HB1615,[Strickenlanguagewouldbedeletedfromandunderlin...
3,AK,HB1156,[Strickenlanguagewouldbedeletedfromandunderlin...
4,AK,SB294,[Strickenlanguagewouldbedeletedfromandunderlin...
...,...,...,...
440,WY,SF111,[2023STATEOFWYOMING23LSO02151SF0111SENATEFILEN...
441,WY,SF144,[2023STATEOFWYOMING23LSO0661ENGROSSED1SF0144SE...
442,WY,HB262,[2023STATEOFWYOMING23LSO05751HB0262HOUSEBILLNO...
443,WY,HB187,[2023STATEOFWYOMING23LSO02521HB0187HOUSEBILLNO...


In [88]:
# Get the bills that are different between the two data frames
list(set(bills_df.bill_name) - set(aclu_table.bill_name))
list(set(aclu_table.bill_name) - set(bills_df.bill_name))

['HB1233']

In [89]:
list(set(aclu_table.bill_name) - set(bills_df.bill_name))

['HB173']

In [64]:
# Merge data frames
merged_df = aclu_table.merge(bills_df, on = ["state", "bill_name"], how = "outer")

In [65]:
merged_df

,state,link,bill_name,category,status,text
0,AK,https://www.akleg.gov/basis/Bill/Detail/33?Roo...,HB27,Schools & Education,"Referred to committee, 01/19/2023",[HB0027a1HB27NewTextUnderlinedDELETEDTEXTBRACK...
1,AK,https://www.akleg.gov/basis/Bill/Detail/33?Roo...,HB105,Schools & Education,"First read and referred to committee, 03/08/2023",[HB0105a1HB105NewTextUnderlinedDELETEDTEXTBRAC...
2,AK,https://www.akleg.gov/basis/Bill/Detail/33?Roo...,SB96,Schools & Education,"First read and referred to committee, 03/08/2023",[SB0096A1SB96NewTextUnderlinedDELETEDTEXTBRACK...
3,AZ,https://apps.azleg.gov/BillStatus/BillOverview...,SB1028,Free Speech & Expression,"Passed Senate; House second read, 03/22/2023",[iSenateEngrossedadultcabaretperformancesprohi...
4,AZ,https://apps.azleg.gov/BillStatus/BillOverview...,SB1026,Free Speech & Expression,"Passed Senate; House second read, 03/09/2023",[iSenateEngrossedstatemoniesdragshowsminorsSta...
...,...,...,...,...,...,...
467,SC,NaN,S274,NaN,NaN,[SouthCarolinaGeneralAssembly125thSession20232...
468,SC,NaN,S627,NaN,NaN,[SouthCarolinaGeneralAssembly125thSession20232...
469,VA,NaN,HB1233,NaN,NaN,[ANACTRelatingtothehousingofinmatesinstatecorr...
470,WA,NaN,SB1203,NaN,NaN,[2023SESSIONINTRODUCED23103055D1SENATEBILLNO12...


In [48]:
# Make a csv file
merged_df.to_csv("../modified_data/merged_bill_data.csv")